In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

# Get Data

In [2]:
# export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
        return map(tensor, (x_train, y_train, x_valid, y_valid))
    
def normalize(x, m, s):
    return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
x_train.mean(), x_train.std()

(tensor(0.0001), tensor(1.))

In [7]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near zero {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max() + 1
n,m,c

(50000, 784, tensor(10))

# Foundations

In [10]:
nh = 50

In [11]:
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
def lin(x, w, b): return x@w1 + b1

In [13]:
t = lin(x_valid, w2, b2)
t.mean(), t.std()

(tensor(-0.2353), tensor(0.9406))

In [14]:
1 - .9987

0.0012999999999999678

In [15]:
def relu(x): return x.clamp_min(0.)

In [16]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.2664), tensor(0.4735))

In [17]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)

In [18]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5863), tensor(0.8983))

In [19]:
def relu(x): return x.clamp_min(0.) - 0.5

In [20]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.0863), tensor(0.8983))

In [21]:
#export
from torch.nn import init

In [22]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0504))

In [23]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.0409), tensor(0.8105))

In [24]:
import torch.nn

In [25]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [26]:
torch.nn.Linear.forward??

In [27]:
torch.nn.functional.linear??

In [28]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [29]:
init.kaiming_normal_??

In [30]:
init.kaiming_uniform_??

In [31]:
w1 = torch.randn(m,nh) * math.sqrt(2./m)

In [32]:
def lin(x, w, b): return x@w + b

# Forward Pass

In [33]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [34]:
%timeit -n 10 _=model(x_valid)

5.38 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
assert model(x_valid).shape == (x_valid.shape[0], 1)

In [36]:
model(x_valid).shape

torch.Size([10000, 1])

In [37]:
#export
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [38]:
y_train, y_valid = y_train.float(), y_valid.float()

In [39]:
output = model(x_train)

In [40]:
mse(output, y_train)

tensor(29.9474)

# Backward Pass

In [41]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [42]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [43]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [44]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [45]:
forward_and_backward(x_train, y_train)

In [46]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [47]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [48]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [49]:
loss = forward(xt2, y_train)

In [50]:
loss.backward()

In [51]:
test_near(ig, xt2.grad)
test_near(w1g, w12.grad)
test_near(w2g, w22.grad)
test_near(b1g, b12.grad)
test_near(b2g, b22.grad)

In [52]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g
        

In [53]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [54]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        return (inp.squeeze(-1) - targ).pow(2).mean()
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze(-1) - self.targ).unsqueeze(-1)/self.targ.shape[0]

In [55]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [56]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [57]:
%time loss = model(x_train, y_train)

CPU times: user 151 ms, sys: 4.69 ms, total: 156 ms
Wall time: 27 ms


In [58]:
%time model.backward()

CPU times: user 5.81 s, sys: 4.46 s, total: 10.3 s
Wall time: 9.24 s


In [60]:
test_near(w1g, w1.g)
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [68]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [69]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * self.out.g

In [70]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [71]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [72]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [73]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [74]:
%time loss = model(x_train, y_train)

CPU times: user 216 ms, sys: 8.08 ms, total: 224 ms
Wall time: 40.6 ms


In [75]:
%time model.backward()

CPU times: user 665 ms, sys: 49.2 ms, total: 715 ms
Wall time: 122 ms


In [76]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [77]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [78]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [79]:
%time loss = model(x_train, y_train)

CPU times: user 167 ms, sys: 1.62 ms, total: 168 ms
Wall time: 29.1 ms


In [80]:
%time model.backward()

CPU times: user 626 ms, sys: 47.6 ms, total: 673 ms
Wall time: 114 ms


In [81]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [82]:
# export
from torch import nn

In [85]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [86]:
model = Model(m, nh, 1)

In [87]:
%time loss = model(x_train, y_train)

CPU times: user 157 ms, sys: 8.24 ms, total: 166 ms
Wall time: 36.2 ms


In [88]:
%time loss.backward()

CPU times: user 222 ms, sys: 11.8 ms, total: 233 ms
Wall time: 43.1 ms


In [93]:
! ./notebook2script.py 02_fully_connected.ipynb

/bin/sh: ./notebook2script.py: Permission denied


In [92]:
!pip install fire

    100% |████████████████████████████████| 81kB 256kB/s ta 0:00:011
  Stored in directory: /Users/a.h.sharma/Library/Caches/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire
